In [1]:
#from __future__ import division, print_function
import matplotlib.pyplot as plt
import bilby
import numpy as np

import time

from bilby.core.prior import Uniform
#from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters, generate_all_bbh_parameters
%matplotlib inline

import hphc7

C = 299792458.
G = 6.67408*1e-11
Mo = 1.989*1e30
Mpc = 3.086*1e22

In [2]:
outdir = 'outdir_new_model'
label = 'create_your_own_source_model'
sampling_frequency = 4096
duration = 1

In [3]:
def eccentric_waveform(frequency_array, mass_1, mass_2, eccentricity, luminosity_distance, theta_jn, psi, phase, geocent_time, ra, dec, **kwargs):
    
    N = len(frequency_array)
    h_plus = np.zeros(N)*1j
    h_cross = np.zeros(N)*1j
    
    fmin = 20.0
    fmax = (C**3)/( G*(mass_1+mass_2)*Mo*np.pi*6**(3/2) )
    
    k = 0
    for f in frequency_array:
        if f>=fmin and f<=fmax:        
            arg_plus = {'iota_':theta_jn, 'beta_':psi, 'D_':luminosity_distance , 'm1_':mass_1, 'm2_':mass_2, 'f_':f, 'f0_':20.0, 'Fp_':1.0, 'Fc_':0.0, 'et0_':eccentricity, 'phic_':phase, 'tc_':geocent_time}

            fplus = hphc7.Fn(**arg_plus)
            
            h_plus[k] = fplus.htilde()[0]
            h_cross[k] = fplus.htilde()[1]
            
        k=k+1

    return {'plus': h_plus, 'cross': h_cross}

In [ ]:
test_frequency = np.arange(20,100,1)
t0 = time.time()
data = eccentric_waveform(test_frequency, 35.0 ,30.0 , 0.1, 440.0, 0.4, 0.1, 1.2, 1180002601.0, 1.375, -1.2108)
t1 = time.time()
print('Execution took {:.4f}'.format(t1 - t0))

In [4]:
# We now define some parameters that we will inject
injection_parameters = dict(mass_1=35.0, mass_2=30.0, eccentricity=0.1, luminosity_distance=440.0, theta_jn=0.4, psi=0.1, phase=1.2, geocent_time=1180002601.0, ra=1.375, dec=-1.2108)

In [5]:
# Now we pass our source function to the WaveformGenerator
waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    frequency_domain_source_model=eccentric_waveform)

22:27 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: __main__.eccentric_waveform
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [6]:
# Set up interferometers.
minimum_frequency = 20.0+5
MM1 = 35.0
MM2 = 30.0
maximum_frequency = (C**3)/( G*(MM1+MM2)*Mo*np.pi*6**(3/2) )-5

ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
for ifo in ifos:
    ifo.minimum_frequency = minimum_frequency
    ifo.maximum_frequency = maximum_frequency
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 3)
ifos.inject_signal(waveform_generator=waveform_generator,
                   parameters=injection_parameters)

/home/hemantakumar.phurailatpam/anaconda3/envs/bilby/lib/python3.7/site-packages/bilby/gw/detector/psd.py:356: RuntimeWarning: invalid value encountered in multiply
  frequency_domain_strain = self.__power_spectral_density_interpolated(frequencies) ** 0.5 * white_noise
/home/hemantakumar.phurailatpam/bilby/0PN/hphc7.py:63: RuntimeWarning: divide by zero encountered in double_scalars
  k = (xk**2 * (27/2 - 7*eta + et0**6 * (1589015535/(5175296*chi**7) - (110496315*eta)/(184832*chi**7) + 498132560585/(1614692352*chi**(19/3)) + (168333575*eta)/(1478656*chi**(19/3)) - 908911888607/(1816528896*chi**(44/9)) + (4154545993*eta)/(4990464*chi**(44/9)) - 892815840919/(1816528896*chi**(38/9)) - (301299733*eta)/(4990464*chi**(38/9)) + 3110697122471/(14532231168*chi**(25/9)) - (12075357445*eta)/(39923712*chi**(25/9)) + 17781607555/(69866496*chi**(19/9)) - (82799465*eta)/(2495232*chi**(19/9))) + et0**4 * ( - (6830363/(153216*chi**(44/9))) + (474967*eta)/(5472*chi**(44/9)) - 268677653/(3983616*chi**(3

ZeroDivisionError: float division by zero

19:54 bilby INFO    : Injected signal in H1:
19:54 bilby INFO    :   optimal SNR = 49.69
19:54 bilby INFO    :   matched filter SNR = 99.34-0.89j
19:54 bilby INFO    :   mass_1 = 35.0
19:54 bilby INFO    :   mass_2 = 30.0
19:54 bilby INFO    :   eccentricity = 0.1
19:54 bilby INFO    :   luminosity_distance = 440.0
19:54 bilby INFO    :   theta_jn = 0.4
19:54 bilby INFO    :   psi = 0.1
19:54 bilby INFO    :   phase = 1.2
19:54 bilby INFO    :   geocent_time = 1180002601.0
19:54 bilby INFO    :   ra = 1.375
19:54 bilby INFO    :   dec = -1.2108
19:54 bilby INFO    : Injected signal in L1:
19:54 bilby INFO    :   optimal SNR = 39.29
19:54 bilby INFO    :   matched filter SNR = 78.31-0.17j
19:54 bilby INFO    :   mass_1 = 35.0
19:54 bilby INFO    :   mass_2 = 30.0
19:54 bilby INFO    :   eccentricity = 0.1
19:54 bilby INFO    :   luminosity_distance = 440.0
19:54 bilby INFO    :   theta_jn = 0.4
19:54 bilby INFO    :   psi = 0.1
19:54 bilby INFO    :   phase = 1.2
19:54 bilby INFO    :  

In [37]:
injection = ifos.inject_signal(waveform_generator=waveform_generator,
                   parameters=injection_parameters)
H1 = ifos[0]
H1_injection = injection[0]

fig, ax = plt.subplots()
idxs = H1.strain_data.frequency_mask  # This is a boolean mask of the frequencies which we'll use in the analysis
ax.loglog(H1.strain_data.frequency_array[idxs],
          np.abs(H1.strain_data.frequency_domain_strain[idxs]),
          label="data")
ax.loglog(H1.frequency_array[idxs],
          H1.amplitude_spectral_density_array[idxs],
          label="ASD")
ax.loglog(H1.frequency_array[idxs],
          np.abs(H1_injection["plus"][idxs]),
          label="Abs. val. of plus polarization")

ax.set_xlim(minimum_frequency,maximum_frequency)
ax.set_ylim(1e-26,2.327737159425905e-21)


ax.set_xlabel("Frequency [Hz]")
ax.set_ylabel("Strain [strain/$\sqrt{Hz}$]")
ax.legend()
plt.show()

In [39]:
#injection_parameters = dict(mass_1=35.0, mass_2=30.0, eccentricity=0.1, luminosity_distance=440.0, 
#theta_jn=0.4, psi=0.1, phase=1.2, geocent_time=1180002601.0, ra=1.375, dec=-1.2108)
priors = bilby.core.prior.PriorDict()
priors["mass_1"] = 35.
priors["mass_2"] = 30.
priors["eccentricity"] = bilby.core.prior.LogUniform(
    name='eccentricity', latex_label='$e$', minimum=0.01, maximum=0.4)
priors["luminosity_distance"] = 440.
priors["theta_jn"] = 0.4
priors["psi"] = 0.1
priors["phase"] = 1.2
priors["geocent_time"] = 1180002601.0
priors["ra"] = 1.375
priors["dec"] = -1.2108


In [41]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=ifos, waveform_generator=waveform_generator)

In [ ]:
result = bilby.core.sampler.run_sampler(
    likelihood, priors, sampler='dynesty', outdir=outdir, label=label,
    resume=False, sample='unif', injection_parameters=injection_parameters)


19:56 bilby INFO    : Running for label 'create_your_own_source_model', output will be saved to 'outdir_new_model'
19:56 bilby INFO    : Using lal version 6.21.0
19:56 bilby INFO    : Using lal git version Branch: None;Tag: lal-v6.21.0;Id: 37caf89936b637993c30b3076fe4b853311873ee;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
19:56 bilby INFO    : Using lalsimulation version 1.10.0
19:56 bilby INFO    : Using lalsimulation git version Branch: None;Tag: lalsimulation-v1.10.0;Id: 5bfe091197594d55cf634f8a1c004eb9dc9b6459;;Builder: Adam Mercer <adam.mercer@ligo.org>;Repository status: CLEAN: All modifications committed
19:56 bilby INFO    : Search parameters:
19:56 bilby INFO    :   eccentricity = LogUniform(minimum=0.01, maximum=0.4, name='eccentricity', latex_label='$e$', unit=None, boundary=None)
19:56 bilby INFO    :   mass_1 = 35.0
19:56 bilby INFO    :   mass_2 = 30.0
19:56 bilby INFO    :   luminosity_distance = 440.0
19:56 bilby I

0it [00:00, ?it/s]

20:10 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'unif', 'verbose': True, 'periodic': None, 'reflective': None, 'check_point_delta_t': 600, 'nlive': 1000, 'first_update': None, 'walks': 100, 'npdim': None, 'rstate': None, 'queue_size': 1, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': 1.5, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 8.0, 'facc': 0.2, 'slices': 5, 'update_interval': 600, 'print_func': <bound method Dynesty._print_func of <bilby.core.sampler.dynesty.Dynesty object at 0x7f1591a51650>>, 'dlogz': 0.1, 'maxiter': None, 'maxcall': None, 'logl_max': inf, 'add_live': True, 'print_progress': True, 'save_bounds': False, 'n_effective': None, 'maxmcmc': 5000, 'nact': 5}
20:10 bilby INFO    : Checkpoint every check_point_delta_t = 600s
20:10 bilby INFO    : Using dynesty version 1.0.1
20:10 bilby INFO    : Generating initial points from the pri

In [17]:
len(result_short.samples)

1779